有同学最近询问如何定位自己感兴趣的城市的图像坐标范围（行列号）。教案中 Figure 6 处的上海区域是通过桌面软件 ENVI 进行坐标确定。作业题中要求的北京、广州的图像坐标，一并附在作业提示中（感谢秋慕冬同学）。

那么，如果我们想裁剪其他感兴趣的城市时，如何通过 Python 来确定遥感图像坐标呢？

借助 GDAL，也可以实现。

以下代码主要借鉴[这篇文章](https://cloud.tencent.com/developer/article/1386146)（感谢作者休斯顿理工大学扫地僧卡尔曼和玻尔兹曼谁曼的开源分享）。

In [7]:
# 🐋：点击右上角⭐即可收藏本cell，以后即可在左侧边栏⭐代码片段库快速调用
# 将经纬度地理坐标转为遥感图上坐标
# -*- encoding: utf-8 -*-

from osgeo import gdal
from osgeo import osr
import numpy as np


def geo2imagexy(dataset, x, y):
    '''
    根据GDAL的六参数模型将给定的投影或地理坐标转为影像图像坐标（行列号）
    :param dataset: GDAL地理数据
    :param x: 投影或地理坐标x
    :param y: 投影或地理坐标y
    :return: r行号,c列号
    '''
    trans = dataset.GetGeoTransform()
    c = (x - trans[0]) / trans[1]
    r = (y - trans[3]) / trans[5]
    return int(r), int(c)


if __name__ == '__main__':
    gdal.AllRegister()
    dataset = gdal.Open(
        r"/home/mw/work/2. clipped dataset/F101992.v4b_web.stable_lights.avg_vis.tif")
    print('数据投影：')
    print(dataset.GetProjection())
    print('数据的大小（行，列）：')
    print('(%s %s)' % (dataset.RasterYSize, dataset.RasterXSize))

    x1 = 120.51  # 上海区域最西边经度
    y1 = 31.53  # 上海区域最北边纬度

    x2 = 122.12  # 上海区域最东边经度
    y2 = 30.40  # 上海区域最南边纬度

    print('左上角地理坐标 -> 图像坐标：')
    coords_1 = geo2imagexy(dataset, x1, y1)
    print('(%s, %s)->(%s, %s)' % (x1, y1, coords_1[0], coords_1[1]))

    print('右下角地理坐标 -> 图像坐标：')
    coords_2 = geo2imagexy(dataset, x2, y2)
    print('(%s, %s)->(%s, %s)' % (x2, y2, coords_2[0], coords_2[1]))

数据投影：

数据的大小（行，列）：
(4538 4012)
左上角地理坐标 -> 图像坐标：
(120.51, 31.53)->(2783, 1991)
右下角地理坐标 -> 图像坐标：
(122.12, 30.4)->(2919, 2184)


👆 简单解释一下转换过程：

c 那行（`c = (x - trans[0]) / trans[1]`），经度 x 减去图像左上角（起始点）经度，得到经度差距，除以经度方向的分辨率（单位也是度），就得到了经度方向的像元个数，即列数。

r 同理（`r = (y - trans[3]) / trans[5]`）。

不懂也没事，下回知道怎么用就行。

PS：为啥输出的 (2783, 1991) 、 (2919, 2184) 跟 Figure 6 中的 (2757, 2012)、(2919, 2190) 不太一样？因为我当初在桌面软件里划的时候没有划很精准……

秋慕冬同学补充：tif 文件一般自带了经纬度信息。地理区域的定位精确与否很大程度受限制于网格的分辨率。比如 0.25\*0.25 格网，经纬度仅仅是一个点，搜索就是找包含这个点的网格，往往出来的范围就大了。如果是要找自然政区不规则的点就相对复杂些，矩形区域还是简单的，所以偏差是正常的。